* 점수제를 통해서 어떤 멀티턴이든 점수를 높이려고 하는 하드 코딩 문제 - 일반 대화 X
* 모듈화 X - NaiveRAG
* Retrieval 고도화 X
* llm 판단 하, 사용자선호도 업데이트 로직

In [1]:
import os
import re
import time
import pickle
import requests
import json
import pandas as pd
import bs4
import tiktoken
import getpass
from tqdm import tqdm
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import clear_output

# 추가된 모듈 임포트
from dotenv import load_dotenv
from pymilvus import connections, utility

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from langchain.vectorstores import Milvus
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# ---------------------- 환경설정 및 임베딩 ----------------------
load_dotenv()

os.environ["NCP_CLOVASTUDIO_API_KEY"] = os.getenv("NCP_CLOVASTUDIO_API_KEY")
os.environ["NCP_CLOVASTUDIO_API_URL"] = os.getenv(
    "NCP_CLOVASTUDIO_API_URL", "https://clovastudio.stream.ntruss.com/"
)

connections.connect(
    alias="default",
    host=os.getenv("MILVUS_HOST", "localhost"),
    port=os.getenv("MILVUS_PORT", "19530"),
)

ncp_embeddings = ClovaXEmbeddings(model="bge-m3")
llm_clova = ChatClovaX(model="HCX-003", max_tokens=2048)

embedding_file = r""
if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"]
    all_metadata_list = saved_data["metadata"]
    print("임베딩 데이터 불러오기")
else:
    raise FileNotFoundError(f"임베딩 파일을 찾을 수 없습니다: {embedding_file}")

metadata_mapping = {
    "ISBN": "ISBN",
    "페이지": "page",
    "가격": "price",
    "제목": "title",
    "저자": "author",
    "분류": "category",
    "저자소개": "author_intro",
    "책소개": "book_intro",
    "목차": "table_of_contents",
    "출판사리뷰": "publisher_review",
    "추천사": "recommendation",
}

all_metadata_list_mapped = []
for meta in all_metadata_list:
    mapped_meta = {metadata_mapping.get(key, key): value for key, value in meta.items()}
    all_metadata_list_mapped.append(mapped_meta)

# embedding 파일 내의 원본 텍스트와 임베딩 데이터를 이용해 문서 객체 생성 (원본 데이터를 외부에서 다시 불러오지 않음)
documents = [
    Document(page_content=pair[0], metadata=meta)
    for pair, meta in zip(all_text_embedding_pairs, all_metadata_list_mapped)
]

collection_name = "book_rag_db"
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

vectorstore = Milvus(
    embedding_function=ncp_embeddings,
    collection_name=collection_name,
    connection_args={"host": "localhost", "port": "19530"},
    auto_id=True,
)

texts = [pair[0] for pair in all_text_embedding_pairs]
embeds = [pair[1] for pair in all_text_embedding_pairs]


# precomputed embedding을 사용하도록 임베딩 메서드 오버라이드
def precomputed_embed_documents(cls, input_texts):
    if input_texts != texts:
        raise ValueError(
            "ERROR : 입력 텍스트 순서가 사전 계산된 임베딩과 일치하지 않음"
        )
    return embeds


ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)
vectorstore.add_texts(
    texts=texts, metadatas=all_metadata_list_mapped, embeddings=embeds
)

임베딩 데이터 불러오기


C:\Users\computer\AppData\Local\Temp\ipykernel_28244\550222839.py:55: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectorstore = Milvus(


[457062786533278377,
 457062786533278378,
 457062786533278379,
 457062786533278380,
 457062786533278381,
 457062786533278382,
 457062786533278383,
 457062786533278384,
 457062786533278385,
 457062786533278386,
 457062786533278387,
 457062786533278388,
 457062786533278389,
 457062786533278390,
 457062786533278391,
 457062786533278392,
 457062786533278393,
 457062786533278394,
 457062786533278395,
 457062786533278396,
 457062786533278397,
 457062786533278398,
 457062786533278399,
 457062786533278400,
 457062786533278401,
 457062786533278402,
 457062786533278403,
 457062786533278404,
 457062786533278405,
 457062786533278406,
 457062786533278407,
 457062786533278408,
 457062786533278409,
 457062786533278410,
 457062786533278411,
 457062786533278412,
 457062786533278413,
 457062786533278414,
 457062786533278415,
 457062786533278416,
 457062786533278417,
 457062786533278418,
 457062786533278419,
 457062786533278420,
 457062786533278421,
 457062786533278422,
 457062786533278423,
 457062786533

In [ ]:
# RetrievalQA 체인 구성 (vectorstore는 precomputed embedding 데이터를 사용)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova, retriever=dense_retriever, return_source_documents=True
)

In [ ]:
# 프롬프트 템플릿 정의
multi_turn_prompt = PromptTemplate(
    input_variables=["history", "query"],
    template="""
[대화 맥락]
사용자 대화 내역:
{history}
사용자의 마지막 질문: "{query}"

[역할]
너는 도서관 사서 챗봇이다. 사용자의 책 추천 요청을 분석하여, 반드시 아래 항목을 출력하라.

[출력 항목]
1. 검색 확률: 0에서 1 사이의 숫자 (예: 0.9)
2. 기본 검색 쿼리: 반드시 유효한 검색 쿼리 텍스트 (예: "한국 고전 소설")
3. 추가 질문: 필요한 경우 추가 정보 요청, 충분하면 빈 문자열

출력 예시:
검색 확률: 0.9
기본 검색 쿼리: "한국 고전 소설"
추가 질문: ""

[지시]
- 대화 내용을 10~20 단어로 요약하고, 위 항목들을 반드시 모두 출력하라.
- "기본 검색 쿼리"가 비어 있거나 "None"이면 안 된다.
""",
)
search_query_chain = LLMChain(llm=llm_clova, prompt=multi_turn_prompt)

final_query_prompt = PromptTemplate(
    input_variables=["history", "fallback"],
    template="""
[대화 요약]
{history}

[기본 검색 쿼리]
{fallback}

위 내용을 바탕으로, 사용자 선호도를 종합하여 DB 검색에 사용할 최종 검색 쿼리를 생성하라.
최종 출력은 반드시 아래 형식을 준수하라:
1. 검색 확률: {{score}}
2. 검색 쿼리: "{{final_search_query}}"
3. 추가 질문: "{{follow_up_question}}"
4. 추천 이유: "{{reason}}"

(추천 이유 정보가 부족하면 "{{reason}}"에는 "추천 이유 정보 없음"이라고 출력하라.)
최종 검색 쿼리:
""",
)
final_query_chain = LLMChain(llm=llm_clova, prompt=final_query_prompt)

C:\Users\computer\AppData\Local\Temp\ipykernel_28244\2251087018.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  search_query_chain = LLMChain(llm=llm_clova, prompt=multi_turn_prompt)


In [ ]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""


MIN_INFO_LENGTH = 10


def generate_answer(query):
    result = dpr_qa_chain.invoke(query)
    source_docs = result["source_documents"]

    # vectorstore (embedding 기반 검색)에서 가져온 문서들의 메타데이터를 활용해 고유 ISBN 목록을 생성
    retrieved_isbns = set()
    for doc in source_docs:
        isbn = doc.metadata.get("ISBN")
        if isbn:
            retrieved_isbns.add(isbn)

    # embedding 파일로부터 생성한 documents 리스트만을 사용하여 ISBN에 해당하는 모든 청크를 집계
    aggregated_docs = []
    for isbn in retrieved_isbns:
        book_docs = [doc for doc in documents if doc.metadata.get("ISBN") == isbn]
        if not book_docs:
            continue
        aggregated_text = "\n".join([doc.page_content for doc in book_docs])
        aggregated_docs.append(
            Document(page_content=aggregated_text, metadata=book_docs[0].metadata)
        )

    formatted_answers = []
    for doc in aggregated_docs:
        metadata = doc.metadata
        title = metadata.get("제목") or extract_field(doc.page_content, "제목")
        author = metadata.get("저자") or extract_field(doc.page_content, "저자")
        publisher_review = extract_field(doc.page_content, "출판사리뷰")
        book_intro = extract_field(doc.page_content, "책소개")

        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""

        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해라. "
                f"사용자 선호도에 기반해서 책의 특징이나 강점을 중심으로 설명해라. 만약 선호도를 모르겠다면, '추천 이유 정보 없음'을 출력해라.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_clova.invoke(reason_prompt)
            # 반환값에서 텍스트 추출 (invoke 결과에 text() 메서드가 있는 경우와 없는 경우 모두 고려)
            if hasattr(reason_response, "text"):
                generated_reason = reason_response.text().strip()
            else:
                generated_reason = str(reason_response).strip()

            if (
                not generated_reason
                or len(generated_reason) < 10
                or "추천 이유 정보 없음" in generated_reason
            ):
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason

        formatted = f"{title}\n{author}\n추천 이유: {reason}"
        formatted_answers.append(formatted)

    answer = "\n\n".join(formatted_answers)
    return answer, None


user_preferences = defaultdict(list)
log_history = []


def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)


def robust_parse_llm_response(response_text):
    score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", response_text)
    search_score = float(score_match.group(1)) if score_match else None

    query_match = re.search(r"기본\s*검색\s*쿼리\s*[:：]\s*\"([^\"]+)\"", response_text)
    search_query = query_match.group(1).strip() if query_match else None

    follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"([^\"]*)\"", response_text)
    follow_up_question = follow_match.group(1).strip() if follow_match else ""

    return search_score, search_query, follow_up_question


def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])
        prompt_vars = {"history": query_summary, "query": query}
        search_decision_dict = search_query_chain.invoke(prompt_vars)
        response_text = search_decision_dict["text"].strip()
        print("\n[LLM 응답 확인]\n", response_text)

        search_score, base_search_query, follow_up_question = robust_parse_llm_response(
            response_text
        )
        print(
            f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'"
        )

        if search_score is None:
            print("\n[LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        if search_score >= 0.8 and base_search_query:
            final_search_query = final_query_chain.invoke(
                {"history": "\n".join(query_history), "fallback": base_search_query}
            )["text"].strip()
            print(f"\n[최종 검색 쿼리 생성]: {final_search_query}")

            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[책 정보]\n{book_info}"
                print("\n[책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info

        if follow_up_question:
            print(f"\n[보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        if search_score < 0.8 or not base_search_query:
            print("\n[검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [ ]:
def interactive_multi_turn_qa():
    query_history = []

    while True:
        clear_output(wait=True)
        print("멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)

        query = input("질문을 입력하세요: ")

        if query.lower() == "quit":
            print("\n[대화 저장 중...]")
            log_history.append(query_history)
            print("대화 저장 완료")
            break

        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)

        print("\n[AI의 답변]")
        print(answer)

        query_history.append(f"AI: {answer}")

        input("\n-> 계속하려면 Enter를 누르세요...")


def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [7]:
# 실행
interactive_multi_turn_qa()

멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------

[대화 저장 중...]
대화 저장 완료
